### What are Markov Chains??

**A Markov Chain is a probabilistic system that provides a way to capture the probability transitions in a system among different states.** 
* Easier way to say this, consider a simple weather tracking system where we have 2 weather patterns only- "R" for Rainy and "S" for Sunny. We have a state space of size = 2, which we will refer to as D = {R,S}. A markob chain for this state space will be a mathematical model that will tell us the probability associated with transition from one state to the other. For a 2-state system, we will observe 4 transitions, R >> R, R >> S, S >> R, and S >> S. 
* Markov chains provide excellent ground for modelling real-world conditions. If independent of the actual phenomena, we would simply assign equal probabilities to each of the transitions. But we know that seldom will a Sunny (S) day be followed by a Rainy (R) only, Sunny weather or any weather pattern tends to prevail for a period before it transitions to the next (lest there be a natural disaster). 
* We can minic this "stickyness" with a two-state Markov chain. When the Markov chain is in state "R", it has a 0.9 probability of staying put and a 0.1 chance of leaving for the "S" state. Likewise, "S" state has 0.9 probability of staying put and a 0.1 chance of transitioning to the "R" state.

We will study the Markov Chains in context of an NLP problem where we often need to construct text sequences which involve predicting the next word given the current word in the sequence. For simplicity all the transitions will be treated as having equal probability of occurance for a given state. For example, if A is a state that can be followed by B,C,D, or E then each transition, A >> B, A >> C, A >> D or A >> E will have equal chance of occuring.

Since, President Trump is yet again being a media hog, I will use the text from his speeches to understand how Markov Chains work.

In [84]:
import urllib.request
import random  

In [85]:
def extract_corpus(text_url):
    words = []
    text = urllib.request.urlopen(text_url)  
    for line in text:
        line = line.decode('utf-8-sig', errors='ignore')
        line = line.strip().lower()
        if line not in ['',' ']:
            new_words = line.split(' ')
            new_words = [word for word in new_words if word not in ['', ' '] and not word.isdigit()]
            words += new_words 
    print('Corpus size: {0} words.'.format(len(words)))
    return words

corpus = extract_corpus(text_url = 'https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt')

Corpus size: 167635 words.


Our next step is to build the **transition probabilities**. We’ll represent our transitions as a dictionary where the keys are the distinct words in the corpus and the value for a given key is a list of words that appear after that key. To build the chain we just need to iterate through the list of words, add it to the dictionary if it’s not already there, and add the word proceeding it to the list of transition words.

In [86]:
def transitions_dict(corpus):
    hashmap = {}
    n_words = len(corpus)
    for idx,key in enumerate(corpus):
        if n_words > idx + 1:
            word = corpus[idx + 1]
            if key not in hashmap:
                hashmap[key] = [word]
            else:
                hashmap[key].append(word)
    print('Chain size: {0} distinct words.'.format(len(hashmap)))
    return hashmap

transition_chain = transitions_dict(corpus)

Chain size: 11221 distinct words.


Though with the above code we are simply adding words to a list for each unique word key in the dictionary, won't it affect the sequence generation later? Answer is Yes, it will impact the generation later but that is desirable because it is simply emulates the phenomena we are trying to observe. If a word occurs multiple times, there’s a higher likelihood that we pick that word proportional to the number of times it appeared after the key relative to all the other words in the corpus that appeared after that key.

In [87]:
def generate_tweet(corpus,transition_chain):
    w1 = random.choice(corpus)  
    tweet = w1
    while len(tweet) < 140:  
        w2 = random.choice(transition_chain[w1])
        tweet += ' ' + w2
        w1 = w2
    print(tweet)
generate_tweet(corpus,transition_chain)

they backed the people that will make our common challenges. for proper instruction from years old, and it’s one company in a real love. and


Now that we know how each unit is going to work, Let's wrap it up together in one main function.

In [88]:
def main():
    corpus = extract_corpus(text_url = 'https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt')
    transition_chain = transitions_dict(corpus)
    generate_tweet(corpus,transition_chain)

if __name__ == "__main__":
    main()

Corpus size: 167635 words.
Chain size: 11221 distinct words.
to south korea. so much — and they’re building an environmental awards from liberty university—do we know we have a company and in business of


We got an understandable, albeit incoherant response for $1^{st}$ order Markov Chains, let us see how the response changes with $2^{nd}$ order Markov Chains. For this we will modify the transition_dict and generate_tweet functions.

In [96]:
def transitions_dict_2nd_order(corpus):
    hashmap = {}
    n_words = len(corpus)
    for idx,key1 in enumerate(corpus):
        if n_words > idx + 1 and n_words > idx + 2:
            key2, word = corpus[idx + 1], corpus[idx + 2]
            if (key1,key2) not in hashmap:
                hashmap[(key1,key2)] = [word]
            else:
                hashmap[(key1,key2)].append(word)
    print('Chain size: {0} distinct words.'.format(len(hashmap)))
    return hashmap

In [97]:
def generate_tweet_2nd_order(corpus,transition_chain,tweet_len=140):
    idx = random.randint(0,len(corpus)-1)  
    key = (corpus[idx],corpus[idx+1])
    tweet = corpus[idx] + ' ' + corpus[idx+1]
    
    while(len(tweet) < tweet_len):
        w = random.choice(transition_chain[key])
        tweet += ' ' + w
        key = (key[1],w)
    print(tweet)

In [98]:
def main():
    corpus = extract_corpus(text_url = 'https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt')
    transition_chain = transitions_dict_2nd_order(corpus)
    for i in range(5):
        print("\n")
        generate_tweet_2nd_order(corpus,transition_chain,tweet_len=200)

if __name__ == "__main__":
    main()

Corpus size: 167635 words.
Chain size: 67132 distinct words.


– they don’t know. so what did she do? she’s destroyed – i said i’ll never run for president i pledge to protect people. we have to be replaced. i don’t think that our people and hispanics working for


to understand that what we want to do this. this is me. look, i always say i’ll be great for the fighter jets. these are the pastors. these are people only believe in each other. here we have to get a


$500 billion trade imbalance – $45 to $50 billion. that’s a good job. we’ll get a lot of things. we are really truly signs of strength. although not in all fairness, in all fairness, has lost 15,000 net


look at what’s happened, and citizens were attacked, as opposed to it now, they wanted to build our military. just the other papers have turned out to be sued, probably will lose the supreme leader of


he doesn’t know what it means. we're going to happen: after i’m called by all of whom are here by the way. i do write about